In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path of folder
folder_path = '/content/drive/MyDrive/Colab Notebooks/2024_08_01 for all sites kmeans site id wise last 3 months /'

# Get list of all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
dfs = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
data = pd.concat(dfs, ignore_index=True)

# Sort by 'First Occurred On' in descending order (latest alarms first)
df = data.sort_values(by='First Occurred On', ascending=False)

# Convert 'First Occurred On' to datetime, handle errors and fill NaT with a default date
df['First Occurred On'] = pd.to_datetime(df['First Occurred On'], errors='coerce')

# Reference time
reference_time = pd.to_datetime('2024-08-01 22:00:00')

# Calculate the relative day index, handle NaT by filling with a large negative number
df['Relative Day Index'] = ((df['First Occurred On'] - reference_time).dt.total_seconds() // 86400).fillna(-9999).astype(int)

# Adjust the index to set the 08:00 to 08:00 of the next day as 0, and previous days as -1, -2, -3, etc.
df['Relative Day Index'] = df['Relative Day Index'].apply(lambda x: x if x < 0 else x)

# Print the updated dataframe
print(df.head(50000))

# Filter the data for HUAWEI and Access domain
filtered_data = df[(df['Vendor'] == 'HUAWEI')]

# Retain only the specified columns
columns_to_keep = ['Site ID', 'Alarm Name', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index']
df_filtered = filtered_data[columns_to_keep]

# Identify the five most recent Site IDs
recent_site_ids = df_filtered['Site ID'].value_counts().index[:10]

# Filter the dataframe to include only the most recent Site IDs
df_filtered = df_filtered[df_filtered['Site ID'].isin(recent_site_ids)]

# Encode categorical columns except 'Relative Day Index'
categorical_columns = df_filtered.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df_filtered[column] = le.fit_transform(df_filtered[column].astype(str))
    label_encoders[column] = le

# Print the updated dataframe
df_filtered.head(100)

Mounted at /content/drive


<ipython-input-1-1ab33db69822>:28: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Clustering to find common patterns
# Determine the optimal number of clusters using the elbow method
# wcss = []
# for i in range(1, 11):
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(df_filtered)
#     wcss.append(kmeans.inertia_)

# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method for Optimal k')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# Fit the KMeans model with the optimal number of clusters
optimal_clusters =20   # Increased number of clusters
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
df_filtered['Cluster'] = kmeans.fit_predict(df_filtered[['Alarm Name', 'Site ID']])

# Now filter for individual site IDs after clustering
for site_id in recent_site_ids:
    past_alarms1 = df_filtered[df_filtered['Site ID'] == site_id]

# Analyze the clusters
print(df_filtered.groupby('Cluster').mean())
print(df_filtered.groupby('Cluster').size())

# Print cluster details and alarm patterns
cluster_patterns = {}
for cluster in range(optimal_clusters):
    print(f"Cluster {cluster} details:")
    cluster_data = df_filtered[df_filtered['Cluster'] == cluster]
    alarm_patterns = cluster_data.groupby(['Alarm Name', 'Site ID']).size()
    cluster_patterns[cluster] = alarm_patterns
    print("Alarm Patterns:")
    print(alarm_patterns)
    print("\n")

# Plot the clusters
# sns.pairplot(df_filtered, hue='Cluster')
# plt.show()

# Inverse transform the scaled columns for interpretation
# scaler = MinMaxScaler()
# df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']] = scaler.inverse_transform(df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']])

# Convert back to original categorical values using label_encoders
# for column in label_encoders.keys():
#     df_filtered[column] = label_encoders[column].inverse_transform(df_filtered[column].round().astype(int))

# Display sample output of clustered data
df_filtered.head(100)

In [ ]:
# Re-scale the dataframe for LSTM
scaler = MinMaxScaler()
df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']] = scaler.fit_transform(df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']])

# Prepare the dataset for LSTM
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10
features = df_filtered[['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster']].values
X, y = create_dataset(features, time_steps)

# Reshape input to be [samples, time steps, features]
X = X.reshape(X.shape[0], time_steps, X.shape[2])

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
model.add(LSTM(50))
model.add(Dense(X.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=1, batch_size=2048, verbose=1)

# Predict for the next 7 days
predictions = []
current_batch = features[-time_steps:].reshape((1, time_steps, X.shape[2]))

for i in range(24 * 7):  # 24 hours * 7 days
    pred = model.predict(current_batch, verbose=0)[0]
    predictions.append(pred)
    current_batch = np.append(current_batch[:, 1:, :], [[pred]], axis=1)

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions)

# Create a DataFrame for predictions
relative_day_indices = np.arange(df_filtered['Relative Day Index'].max() + 1, df_filtered['Relative Day Index'].max() + 1 + 24 * 7)
pred_df = pd.DataFrame(predictions, columns=['Alarm Name', 'Site ID', 'Vendor', 'Domain', 'Device Type', 'Relative Day Index', 'Cluster'])
pred_df['Relative Day Index'] = relative_day_indices

# Convert back to original categorical values using label_encoders
for column in label_encoders.keys():
    pred_df[column] = pred_df[column].round().astype(int)
    pred_df[column] = label_encoders[column].inverse_transform(pred_df[column])

# Filter predictions for the next 7 days only
pred_df = pred_df[pred_df['Relative Day Index'] <= 7]

# Display sample output of predictions with cluster details
print(pred_df.head(7))

# Print the cluster patterns for the predicted alarms
for index, row in pred_df.iterrows():
    cluster = int(row['Cluster'])
    print(f"Predicted Alarm for Relative Day Index {row['Relative Day Index']}:")
    print(f"Cluster {cluster} Alarm Patterns:")
    alarm_names = cluster_patterns[cluster].index.get_level_values(0).map(lambda x: label_encoders['Alarm Name'].inverse_transform([x])[0])
    print(pd.Series(cluster_patterns[cluster].values, index=alarm_names))
    print("\n")

# Print cluster-wise alarm patterns
for cluster in range(optimal_clusters):
    print(f"Cluster {cluster} Alarm Patterns:")
    alarm_names = cluster_patterns[cluster].index.get_level_values(0).map(lambda x: label_encoders['Alarm Name'].inverse_transform([x])[0])
    print(pd.Series(cluster_patterns[cluster].values, index=alarm_names))
    print("\n")

In [ ]:
import matplotlib.pyplot as plt

# Print cluster-wise alarm patterns
for cluster in range(optimal_clusters):
    print(f"Cluster {cluster} Alarm Patterns:")
    alarm_names = cluster_patterns[cluster].index.get_level_values(0).map(lambda x: label_encoders['Alarm Name'].inverse_transform([x])[0])
    alarm_counts = pd.Series(cluster_patterns[cluster].values, index=alarm_names)
    print(alarm_counts)
    print("\n")

    # Plot the alarm patterns for the current cluster
    plt.figure(figsize=(10, 6))  # Adjust figure size as needed
    alarm_counts.plot(kind='bar')
    plt.title(f'Alarm Patterns for Cluster {cluster}')
    plt.xlabel('Alarm Name')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
    plt.tight_layout()  # Adjust layout to prevent labels from overlapping
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Iterate through each site ID
for site_id in recent_site_ids:


    # Filter predicted alarms for the current site
    predicted_alarms = pred_df[pred_df['Site ID'] == site_id]

    # Plot the alarms
    plt.figure(figsize=(12, 6))  # Adjust figure size as needed

    # Plot past alarms
    plt.scatter(past_alarms1['Relative Day Index'], past_alarms1['Alarm Name'],
                c=past_alarms1['Cluster'], cmap='viridis', marker='o', label='Past Alarms', s=50)

    # Plot predicted alarms
    plt.scatter(predicted_alarms['Relative Day Index'], predicted_alarms['Alarm Name'],
                c=predicted_alarms['Cluster'], cmap='viridis', marker='x', label='Predicted Alarms', s=50)

    plt.title(f'Past and Predicted Alarms for Site {site_id}')
    plt.xlabel('Relative Day Index')
    plt.ylabel('Alarm Name')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels if needed
    plt.legend()
    plt.colorbar(label='Cluster')  # Add a colorbar to show cluster mapping
    plt.tight_layout()  # Adjust layout to prevent labels from overlapping
    plt.show()